# Code Setup

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

from utils.import_puzzle_input import load_input
from utils.import_puzzle_input import split_puzzle_input

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload modules before executing code
%autoreload 2

In [2]:
puzzle_input = load_input(2023, 7)

loaded puzzle input


In [3]:
puzzle_input_split = split_puzzle_input(puzzle_input)
puzzle_input_split

['6JA22 162',
 'TQJQ8 732',
 '7T77A 882',
 '6K66K 850',
 'QQAQQ 11',
 '7QQ7Q 321',
 '28966 921',
 '34433 801',
 '8QQ8Q 914',
 'K63K5 636',
 '47777 533',
 'K62Q9 773',
 '25J52 717',
 'A6AAA 631',
 '23222 720',
 '267Q4 9',
 'K8QK5 15',
 'QQA22 400',
 '563T8 50',
 '2J5J2 756',
 '77T83 647',
 'T9596 27',
 '53AJA 371',
 '6J666 916',
 '833TK 380',
 '5TK4J 506',
 'TK3KK 794',
 'JTJ55 54',
 'TTT4T 422',
 '9J3KK 491',
 '488K4 30',
 '87878 654',
 'QQKKK 894',
 'A4A3J 360',
 '55559 232',
 '33388 148',
 'JK599 632',
 'KK663 300',
 'JJ448 514',
 'AKATA 188',
 '6J537 885',
 '98888 65',
 '9TT3J 361',
 '668A9 227',
 'Q3285 257',
 'JT54J 208',
 'K8QA7 373',
 '77K39 681',
 '33T4J 281',
 'KA832 578',
 '6QQQ5 214',
 '52222 154',
 '6A4Q4 78',
 '39339 282',
 '2A82A 25',
 '5A66J 261',
 '5594J 4',
 'TTTT6 752',
 '9A3K5 760',
 'KA5AA 26',
 'K7464 447',
 '33676 605',
 'K7J7A 946',
 '5T5KQ 805',
 '654Q8 888',
 '35358 315',
 'A4A57 552',
 '669AA 485',
 '7A77Q 105',
 'Q3333 372',
 '57A9Q 665',
 '97655 367',
 '2422

# Problem Setup

# General thoughts

This problem sounds like it should be solved with a custom class with comparison method - see datacamp OOP course

# Solutions

## Solution 1:

### Explanation of method

0. We will retrieve the data
1. We will make a function to convert a card hand string to a type strength depending on which kind of type of hand it is (five of a kind, four of a kind, full house, three of a kind, ...)
2. We will make a function that takes a string and goes through the characters of the string and converts each character to its strength, thus converting the string of characters to a list of character strength ratings
3. Take `enumerate` on the ordering of the strings, and join the ranking onto the data
4. Multiply the bid and rank in each case and sum these

#### Remarks

### Implementation

#### Part 0

In [10]:
puzzle_input_data = pd.DataFrame([*map(lambda x: x.split(" "), puzzle_input_split)], columns = ["hand", "bid"])
puzzle_input_data

,hand,bid
0,6JA22,162
1,TQJQ8,732
2,7T77A,882
3,6K66K,850
4,QQAQQ,11
...,...,...
995,KK399,948
996,QJ828,791
997,97697,211
998,KKK6J,719


#### Part 1

In [12]:
def get_type_strength(s):
        count_of_cards = Counter(s)
        count_of_cards.__len__()

        if count_of_cards.__len__() == 1:
            # 'Five of a kind'
            type_strength = 0
        elif count_of_cards.__len__() == 2:
            # 'Four of a Kind', OR, 'Full House'
            if count_of_cards.most_common(n = 1)[0][1] == 4:
                # Four of a kind
                type_strength = 1
            else:
                # Full house
                type_strength = 2
        elif count_of_cards.__len__() == 3:
            # 'Three of a Kind' or 'Two Pair'
            if count_of_cards.most_common(n = 1)[0][1] == 3:
                #' Three of a kind'
                type_strength = 3
            else:
                # 'Two Pair'
                type_strength = 4
        elif count_of_cards.__len__() == 4:
            # 'One Pair'
            type_strength = 5
        elif count_of_cards.__len__() == 5:
            # 'High Card'
            type_strength = 6
        else:
            raise ValueError

        return type_strength

#### Part 2

In [13]:
def sort_hands(hands: list[str], char_order: list[str]):
    #!!! A check on there only being characters in the hands that are of an order recorded in char_order

    char_order_map = {char: int(idx) for idx, char in enumerate(char_order)}

    sorted_hands = sorted(hands, key = lambda s: [get_type_strength(s)] + [char_order_map[c] for c in s])
    return sorted_hands

#### Part 3

In [14]:
char_order = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]

In [23]:
def create_hand_rank_data(puzzle_input_data: pd.DataFrame, char_order: list[str]) -> pd.DataFrame:
    sorted_hands = sort_hands(puzzle_input_data.iloc[:, 0], char_order)
    len_sorted_hands = len(sorted_hands)
    
    sorted_hand_rank_data = pd.DataFrame([[s, len_sorted_hands - idx] for idx, s in enumerate(sorted_hands)], columns = ["hand", "rank"])
    merged_frame = pd.merge(puzzle_input_data, sorted_hand_rank_data, how = "left", on = "hand")
    merged_frame['rank'] = merged_frame['rank'].astype(np.int64)
    merged_frame['bid'] = merged_frame['bid'].astype(np.int64)
    return merged_frame

#### Part 4

In [26]:
merged_frame = create_hand_rank_data(puzzle_input_data, char_order)
merged_frame

,hand,bid,rank
0,6JA22,162,287
1,TQJQ8,732,355
2,7T77A,882,698
3,6K66K,850,832
4,QQAQQ,11,984
...,...,...,...
995,KK399,948,599
996,QJ828,791,399
997,97697,211,545
998,KKK6J,719,774


In [28]:
def solve_puzzle(puzzle_input_data: pd.DataFrame, char_order: list[str]):
    merged_data = create_hand_rank_data(puzzle_input_data, char_order)
    puzzle_output = (merged_data["rank"] * merged_data["bid"]).sum()
    return puzzle_output

In [29]:
solve_puzzle(puzzle_input_data, char_order)

np.int64(252052080)

### Checking runtime

#### Benchmarking

In [ ]:
%%timeit
puzzle_input_data_old_01 = np.array(list(map(lambda x: x.split(" "), puzzle_input_split)))
# puzzle_input_data_old_01

1.3 ms ± 81.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
puzzle_input_data_old_015 = np.array(list(map(lambda x: x.split(" "), puzzle_input_split)))
# puzzle_input_data_old_015

1.36 ms ± 248 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
puzzle_input_data_old_02 = list(map(lambda x: x.split(" "), puzzle_input_split))
# puzzle_input_data_old_02

489 μs ± 67.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
puzzle_input_data_old_025 = [(hand, int(bid)) for hand, bid in [*map(lambda x: x.split(" "), puzzle_input_split)]]
# puzzle_input_data_old_025

1.09 ms ± 355 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
puzzle_input_data_old_03 = [(hand, int(bid)) for hand, bid in (x.split(" ") for x in puzzle_input_split)]
# puzzle_input_data_old_03

1.01 ms ± 231 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Runtime profiling

## Solution 2:

# Experimental

In [20]:
test_two_pair = Counter("QKKAA")
test_two_pair

Counter({'K': 2, 'A': 2, 'Q': 1})

In [19]:
test_two_pair.most_common(n = 1)

[('A', 2)]

In [22]:
test_two_pair.most_common(n = 1)[0][1]

2

In [ ]:
len(test_two_pair)

3

In [24]:
test_two_pair.__len__()

3

In [ ]:
test_two_pair.

In [ ]:
help(Counter("blabla"))

In [ ]:
class hand:
    STRENGTH_TO_TYPE_MAP = {
        7: 'Five of a kind',
        6: 'Four of a kind',
        5: 'Full house',
        4: 'Three of a kind',
        3: 'Two pair',
        2: 'One pair',
        1: 'High card',
    }

    def __init__(self, cards: str, bid: int):
        self.cards = cards
        self.bid = bid

    def count(self):
        return Counter(self.cards)
    
    def get_type_strength(self):
        count_of_cards = self.count()
        count_of_cards.__len__()

        if count_of_cards.__len__() == 1:
            # 'Five of a kind'
            type_strength = 6
        elif count_of_cards.__len__() == 2:
            # 'Four of a Kind', OR, 'Full House'
            if count_of_cards.most_common(n = 1)[0][1] == 4:
                # Four of a kind
                type_strength = 5
            else:
                # Full house
                type_strength = 4
        elif count_of_cards.__len__() == 3:
            # 'Three of a Kind' or 'Two Pair'
            if count_of_cards.most_common(n = 1)[0][1] == 3:
                #' Three of a kind'
                type_strength = 3
            else:
                # 'Two Pair'
                type_strength = 2
        elif count_of_cards.__len__() == 4:
            # 'One Pair'
            type_strength = 1
        elif count_of_cards.__len__() == 5:
            # 'High Card'
            type_strength = 0
        else:
            raise ValueError

        return type_strength
    
    def get_position_strength(self):
        pass

    def __str__(self):
        return f"Hand: '{self.cards}'\nType: '{hand.STRENGTH_TO_TYPE_MAP[self.get_type_strength()]}' \nBid: {self.bid}"
                

In [40]:
test_hand_01 = hand("32T3K", 765)

In [41]:
print(test_hand_01)

Hand: '32T3K'
Type: 'One pair' 
Bid: 765


In [27]:
test_hand_01.get_type_strength()

2

In [5]:
test_char_order = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]

In [13]:
test_order_map_smallest = {char: idx for idx, char in enumerate(test_char_order)}
test_order_map_smallest

{'A': 0,
 'K': 1,
 'Q': 2,
 'J': 3,
 'T': 4,
 '9': 5,
 '8': 6,
 '7': 7,
 '6': 8,
 '5': 9,
 '4': 10,
 '3': 11,
 '2': 12}

In [11]:
str(12-0)

'12'

In [14]:
test_order_map_largest = {char: 12 - int(idx) for idx, char in enumerate(test_char_order)}
test_order_map_largest

{'A': 12,
 'K': 11,
 'Q': 10,
 'J': 9,
 'T': 8,
 '9': 7,
 '8': 6,
 '7': 5,
 '6': 4,
 '5': 3,
 '4': 2,
 '3': 1,
 '2': 0}

I think this might be a good opportunity to learn algorithms and data structures for sorting.

In [42]:
from functools import cmp_to_key

In [43]:
help(cmp_to_key)

Help on built-in function cmp_to_key in module _functools:

cmp_to_key(mycmp)
    Convert a cmp= function into a key= function.

    mycmp
      Function that compares two objects.



In [44]:
order = 'ABCDEFGHIJKLM'
def custom_cmp(a, b):
    return order.index(a) - order.index(b)

In [49]:
a_tmp = "D"
b_tmp = "F"
tmp_cmp_to_key = cmp_to_key(custom_cmp(a_tmp, b_tmp))

In [56]:
tmp_cmp_to_key

In [15]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.

    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



In [23]:
def get_type_strength(s):
        count_of_cards = Counter(s)
        count_of_cards.__len__()

        if count_of_cards.__len__() == 1:
            # 'Five of a kind'
            type_strength = 6
        elif count_of_cards.__len__() == 2:
            # 'Four of a Kind', OR, 'Full House'
            if count_of_cards.most_common(n = 1)[0][1] == 4:
                # Four of a kind
                type_strength = 5
            else:
                # Full house
                type_strength = 4
        elif count_of_cards.__len__() == 3:
            # 'Three of a Kind' or 'Two Pair'
            if count_of_cards.most_common(n = 1)[0][1] == 3:
                #' Three of a kind'
                type_strength = 3
            else:
                # 'Two Pair'
                type_strength = 2
        elif count_of_cards.__len__() == 4:
            # 'One Pair'
            type_strength = 1
        elif count_of_cards.__len__() == 5:
            # 'High Card'
            type_strength = 0
        else:
            raise ValueError

        return type_strength
    

In [45]:
def custom_sorting(strings: list[str], char_order: list[str]):
    #!!! A check on there only being characters in the strings that are of an order recorded in char_order
    len_char_order = len(char_order)
    char_order_map = {char: len_char_order - 1 - int(idx) for idx, char in enumerate(char_order)}

    # sorted(strings, key = lambda s: tuple(get_type_strength(s), char_order_map[c] for c in s), reverse = True)
    reverse_sorted_strings = sorted(strings, key = lambda s: [get_type_strength(s)] + [char_order_map[c] for c in s], reverse = True)
    return reverse_sorted_strings

In [46]:
test_list_of_str = ["32T3K","T55J5","KK67","KTJJT","QQQJA"]

In [47]:
custom_sorting(test_list_of_str, test_char_order)

['QQQJA', 'T55J5', 'KK67', 'KTJJT', '32T3K']

In [38]:
test_string = "32T3K"

In [39]:
test_char_order = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]

In [40]:
list(enumerate(test_char_order))

[(0, 'A'),
 (1, 'K'),
 (2, 'Q'),
 (3, 'J'),
 (4, 'T'),
 (5, '9'),
 (6, '8'),
 (7, '7'),
 (8, '6'),
 (9, '5'),
 (10, '4'),
 (11, '3'),
 (12, '2')]

In [41]:
test_len_char_order = len(test_char_order)

In [42]:
test_char_order_map = {char: test_len_char_order - 1 - int(idx) for idx, char in enumerate(test_char_order)}
test_char_order_map

{'A': 12,
 'K': 11,
 'Q': 10,
 'J': 9,
 'T': 8,
 '9': 7,
 '8': 6,
 '7': 5,
 '6': 4,
 '5': 3,
 '4': 2,
 '3': 1,
 '2': 0}

In [43]:
[get_type_strength(test_string)] + [test_char_order_map[c] for c in test_string]

[1, 1, 0, 8, 1, 11]

In [44]:
test_string

'32T3K'

## Get unique characters of a list of strings

In [70]:
test_string = "32T3K"
test_list_of_str = ["32T3K","T55J5","KK67","KTJJT","QQQJA"]

In [75]:
list(map(set, test_list_of_str))

[{'2', '3', 'K', 'T'},
 {'5', 'J', 'T'},
 {'6', '7', 'K'},
 {'J', 'K', 'T'},
 {'A', 'J', 'Q'}]

In [76]:
set.union(*map(set, test_list_of_str))

{'2', '3', '5', '6', '7', 'A', 'J', 'K', 'Q', 'T'}

In [ ]:
def check_allowed_characters(strings: list[str], allowed_chars: set):
    map_of_chars = map(set, strings)
    unique_chars_in_strings = set.union(*map_of_chars)

    chars_in_strings_not_in_allowed_chars = unique_chars_in_strings - allowed_chars

    if len(chars_in_strings_not_in_allowed_chars) != 0:
        raise ValueError("'strings' contains characters not allowed as per the 'allowed_chars' specification")

### Add a column to a array

array([['6JA22', '162'],
       ['TQJQ8', '732'],
       ['7T77A', '882'],
       ...,
       ['97697', '211'],
       ['KKK6J', '719'],
       ['AA5TT', '937']], shape=(1000, 2), dtype='<U5')